Importing all needed libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.tree import DecisionTreeRegressor
import numpy as np

import optuna

c:\Users\esian\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Testing NN ml

In [2]:
file_path = r"C:\Users\esian\Desktop\Kafe\src\model\merged_data.csv"
df = pd.read_csv(file_path)
encoder = LabelEncoder()
df['item_encoded'] = encoder.fit_transform(df['item'])

X = df[['day_of_week', 'is_holiday', 'temperature_2m_max', 
             'temperature_2m_min', 'precipitation_sum', 'item_encoded']]  
y = df['sales'].fillna(0)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, train_size=0.8, random_state=42, shuffle=True
)

# -----------------------------
# Optuna objective function
# -----------------------------
def objective(trial):
    # Hyperparameters to search
    n_units = trial.suggest_int("n_units", 16, 256)
    n_layers = trial.suggest_int("n_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    # Build model
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    for i in range(n_layers):
        model.add(layers.Dense(n_units, activation="relu"))
        model.add(layers.Dropout(dropout))

    model.add(layers.Dense(1, activation="linear"))  # regression output

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=["mae"]
    )

    # Train model
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=20,  # keep small for tuning
        batch_size=32,
        verbose=0
    )

    # Return the last validation MAE
    val_mae = history.history["val_mae"][-1]
    return val_mae

# -----------------------------
# Run Optuna optimization
# -----------------------------
study = optuna.create_study(direction="minimize")  # minimize MAE
study.optimize(objective, n_trials=20)

print("Best params:", study.best_params)

# -----------------------------
# Train final model with best params
# -----------------------------
best_params = study.best_params
final_model = keras.Sequential()
final_model.add(layers.Input(shape=(X_train.shape[1],)))

for i in range(best_params["n_layers"]):
    final_model.add(layers.Dense(best_params["n_units"], activation="relu"))
    final_model.add(layers.Dropout(best_params["dropout"]))

final_model.add(layers.Dense(1, activation="sigmoid"))

final_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=best_params["learning_rate"]),
    loss="mse",
    metrics=["mae"]
)

final_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16)

# Evaluate
test_loss, test_mae = final_model.evaluate(X_test, y_test, verbose=0)
print(f"Final Test MAE: {test_mae}, {test_loss}")



[I 2025-09-10 19:55:19,338] A new study created in memory with name: no-name-431a4bcc-2ddf-41c8-a9a3-ef7860d5bb60
[I 2025-09-10 19:55:30,776] Trial 0 finished with value: 4.914270877838135 and parameters: {'n_units': 141, 'n_layers': 3, 'dropout': 0.3464972793912885, 'learning_rate': 0.003629910797814957}. Best is trial 0 with value: 4.914270877838135.
[I 2025-09-10 19:55:39,291] Trial 1 finished with value: 4.915328025817871 and parameters: {'n_units': 41, 'n_layers': 2, 'dropout': 0.35968768274406915, 'learning_rate': 0.0008820742278926103}. Best is trial 0 with value: 4.914270877838135.
[I 2025-09-10 19:55:45,589] Trial 2 finished with value: 8.525769233703613 and parameters: {'n_units': 131, 'n_layers': 1, 'dropout': 0.14082252733475614, 'learning_rate': 0.0012689123440447856}. Best is trial 0 with value: 4.914270877838135.
[I 2025-09-10 19:55:57,434] Trial 3 finished with value: 4.9405341148376465 and parameters: {'n_units': 159, 'n_layers': 2, 'dropout': 0.1454812963931277, 'lear

Best params: {'n_units': 21, 'n_layers': 2, 'dropout': 0.4744782082801334, 'learning_rate': 0.009490461859535842}
Epoch 1/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - loss: 104.7632 - mae: 8.2037 - val_loss: 103.9238 - val_mae: 8.1851
Epoch 2/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 102.8513 - mae: 8.0899 - val_loss: 103.8718 - val_mae: 8.1820
Epoch 3/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 102.7980 - mae: 8.0874 - val_loss: 103.8634 - val_mae: 8.1815
Epoch 4/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 102.7846 - mae: 8.0863 - val_loss: 103.8610 - val_mae: 8.1813
Epoch 5/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 102.7732 - mae: 8.0857 - val_loss: 103.8601 - val_mae: 8.1813
Epoch 6/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 102.7690 - mae: 8.0854 - val_loss: 103.8598 - val_mae: 8.1813
Epoch 7/50
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 102.7680 - mae: 8.0852 - val_loss: 103.8596 - val_mae: 8.1813
Epoch 8/50
100/100 ━━━━━━━━━━━━━

Testing Random Forest

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings


from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error


warnings.filterwarnings('ignore')

# open sample data fiel
file_path = (r"C:\Users\esian\Desktop\Kafe\src\model\merged_data.csv")
df = pd.read_csv(file_path)

# Encode item column
encoder = LabelEncoder()
df['item_encoded'] = encoder.fit_transform(df['item'])

# set features
X = df[['day_of_week', 'is_holiday','sales_lag_1', 'sales_lag_2', 'sales_lag_3', 
        'sales_lag_7','temperature_2m_max', 'temperature_2m_min', 'precipitation_sum','item_encoded']]  

# check shape
print(X.shape, y.shape) 
y = y.fillna(0) 

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42, shuffle=True)

# Create Random Forest regressor
regressor = RandomForestRegressor(n_estimators=100, random_state=42, oob_score=True)

# Train the model
regressor.fit(X_train, y_train)

# Predict
y_pred = regressor.predict(X_test)

# Evaluate
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error:", mae)


(1997, 10) (1997,)
Mean Squared Error: 23.656068021944442
Mean Absolute Error: 3.8844558333333334
